In [1]:
import pandas as pd
import numpy as np
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfTransformer
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import random
import tensorflow as tf

In [2]:
# Nhập dữ liệu
df = pd.read_csv('./data/product_df.csv')
df = df[['Star Rating', 'Comment']]

In [3]:
def label_sentiment(rating):
    if rating in [1, 2]:
        return '0'
    elif rating == 3:
        return '1'
    elif rating in [4, 5]:
        return '2'
    else:
        return '3'  # Nếu có xếp hạng nằm ngoài khoảng 1-5

# Gắn nhãn cảm xúc cho mỗi đánh giá
df['Sentiment'] = df['Star Rating'].apply(label_sentiment)

# Hiển thị 5 hàng đầu tiên của dataframe với cột sentiment mới
print(df.head())

   Star Rating                                            Comment Sentiment
0            4                      Điện thoại này dùng rất thích         2
1            4                               sử dụng thấy cũng ok         2
2            2                      Bảo hành ít quá, chỉ 12 tháng         0
3            5                             Sản phẩm mượt, chạy êm         2
4            3  Cho mình hỏi muốn khởi động lại máy hay tắt ng...         1


In [4]:
# Kiểm tra và loại bỏ giá trị khuyết thiếu
print(df.isnull().sum())
df = df.dropna(subset=['Comment'])
# Kiểm tra và loại bỏ giá trị khuyết thiếu
print(df.isnull().sum())
df = df.dropna(subset=['Comment'])
# Kiểm tra và loại bỏ dữ liệu trùng lặp
duplicate_comments = df[df.duplicated(['Comment'])]
print("Các dòng dữ liệu trùng lặp trong cột 'Comment':")
print(duplicate_comments)
df = df.drop_duplicates(['Comment'])
print("Shape after dropping duplicates:", df.shape)

Star Rating    0
Comment        0
Sentiment      0
dtype: int64
Star Rating    0
Comment        0
Sentiment      0
dtype: int64
Các dòng dữ liệu trùng lặp trong cột 'Comment':
      Star Rating                                   Comment Sentiment
48              4                                   Rất tốt         2
76              5                                    Rất ok         2
107             2                             pin tụt nhanh         0
135             5                              sản phẩm tốt         2
173             2                             hao pin nhanh         0
177             5                              sản phẩm tốt         2
181             5                                    Rất ok         2
183             5                              sản phẩm tốt         2
192             5                               sản phẩm ok         2
205             5                               máy dùng ok         2
225             4                                     

In [5]:
# Chuẩn hóa và làm sạch văn bản
def remove_special_characters(text):
    # Loại bỏ các ký tự đặc biệt, giữ lại chữ cái, số, và các dấu câu
    return re.sub(r'[^a-zA-ZÀ-ỹà-ỹ0-9\s]', '', text)

def to_lowercase(text):
    # Chuyển đổi văn bản về chữ thường
    return text.lower()

def normalize_text(text):
    text = remove_special_characters(text)
    text = to_lowercase(text)
    return text

df['Comment'] = df['Comment'].apply(normalize_text)
print(df.head())

   Star Rating                                            Comment Sentiment
0            4                      điện thoại này dùng rất thích         2
1            4                               sử dụng thấy cũng ok         2
2            2                       bảo hành ít quá chỉ 12 tháng         0
3            5                              sản phẩm mượt chạy êm         2
4            3  cho mình hỏi muốn khởi động lại máy hay tắt ng...         1


In [6]:
# Lưu đánh giá vào cột X
X = df['Comment'].values.tolist()
# Lưu nhãn vào cột Y
y = df['Sentiment'].values.tolist()

In [7]:
#Xây dựng hàm token từ underthesea
def tokenize_and_build_vocab_vietnamese(comment):
    tokens = word_tokenize(comment, format="text")
    return tokens.split()

In [8]:
vectorizer = TfidfVectorizer(tokenizer=tokenize_and_build_vocab_vietnamese, token_pattern=None,
                              max_features=5000, ngram_range=(1, 2), max_df=0.85, min_df=5)
# Vectorize dữ liệu
X_tfidf = vectorizer.fit_transform(X)
vocabulary = vectorizer.get_feature_names_out()

In [9]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [10]:
#Sử dụng SMOTE để cân bằng dữ liệu

print('Trước khi cân bằng:', Counter(y))

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

print('Sau khi cân bằng:', Counter(y_resampled))


Trước khi cân bằng: Counter({2: 832, 0: 428, 1: 238})
Sau khi cân bằng: Counter({2: 832, 0: 832, 1: 832})


In [11]:
print('Kích thước dữ liệu ban đầu:', X_tfidf.shape)
print('Kích thước dữ liệu sau khi cân bằng:', X_resampled.shape)
# In ra số lượng mẫu của từng nhãn trước và sau khi cân bằng
print('Trước khi cân bằng:', Counter(y))
print('Sau khi cân bằng:', Counter(y_resampled))

Kích thước dữ liệu ban đầu: (1498, 1200)
Kích thước dữ liệu sau khi cân bằng: (2496, 1200)
Trước khi cân bằng: Counter({2: 832, 0: 428, 1: 238})
Sau khi cân bằng: Counter({2: 832, 0: 832, 1: 832})


In [12]:
# Chia tập dữ liệu thành tập huấn luyện và tập validation (validation + test)
X_train_temp, X_test, y_train_temp, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=42)

# Chia tập tạm thời thành tập validation và tập test cuối cùng
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.2, random_state=42)

In [13]:
# Tạo mô hình Naive Bayes
nb_model = MultinomialNB()

# Định nghĩa các tham số để tinh chỉnh
nb_params = {
    'alpha': [0.1, 0.5, 1.0],
    'fit_prior': [True, False],
}

# Tinh chỉnh hyperparameters bằng GridSearchCV
nb_grid = GridSearchCV(estimator=nb_model, param_grid=nb_params, cv=3, scoring='accuracy')
nb_grid.fit(X_train, y_train)

# Lấy mô hình tốt nhất sau khi tinh chỉnh
best_nb = nb_grid.best_estimator_

In [14]:
# Tạo mô hình SVM
svm_model = SVC()

# Định nghĩa các tham số để tinh chỉnh
svm_params = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
}

# Tinh chỉnh hyperparameters bằng GridSearchCV
svm_grid = GridSearchCV(estimator=svm_model, param_grid=svm_params, cv=3, scoring='accuracy')
svm_grid.fit(X_train, y_train)

# Lấy mô hình tốt nhất sau khi tinh chỉnh
best_svm = svm_grid.best_estimator_

In [15]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Hàm xây dựng mô hình cho GridSearch
def create_model(neurons, dropout_rate, optimizer):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(neurons // 2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Xây dựng mô hình KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=0)

# Tham số tìm kiếm GridSearch
param_grid = {
    'batch_size': [32, 64, 128],
    'epochs': [5,10, 20, 30],
    'optimizer': ['adam', 'rmsprop'],
    'neurons': [64, 128],
    'dropout_rate': [0.2, 0.3]
}

# GridSearch
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Lấy mô hình tốt nhất sau khi tinh chỉnh
best_dnn = grid_result.best_estimator_

C:\Users\vuhamy\AppData\Local\Temp\ipykernel_12604\1867992275.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


In [ ]:
from sklearn.ensemble import VotingClassifier

# Define the classifiers with their best parameters
nb_model = best_nb
svm_model = best_svm
dnn_model = best_dnn.model
# Create a voting classifier with soft voting
voting_classifier = VotingClassifier(
    estimators=[('nb', nb_model), ('svm', svm_model), ('dnn', dnn_model)],
    voting='soft'  # Use 'soft' for probabilities
)

# Fit the voting classifier on the training data
voting_classifier.fit(X_train, y_train)

# Predict on the validation set
y_pred = voting_classifier.predict(X_val)

In [ ]:
from sklearn.ensemble import VotingClassifier

# Kết hợp các mô hình vào một Voting Classifier
voting_classifier = VotingClassifier(estimators=[
    ('nb', best_nb),  # Naive Bayes đã tinh chỉnh
    ('svm', best_svm),  # SVM đã tinh chỉnh
    ('dnn', best_dnn)  # DNN đã tinh chỉnh
], voting='soft')  # Sử dụng soft voting

# Huấn luyện mô hình kết hợp
voting_classifier.fit(X_train, y_train)

# Đánh giá mô hình kết hợp trên tập kiểm tra
y_pred = voting_classifier.predict(X_test)
print(classification_report(y_test, y_pred))


NameError: name 'best_nb' is not defined

In [ ]:
# Bao gồm comment mới vào một list
new_comments = [
    "Sản phẩm tốt, rất hài lòng!",
    "Pin yếu, cam tệ, màn hình giật lag liên tục",
    "Chỉ sử dụng ở mức tạm ổn, tôi muốn pass lại"
]

# Chuẩn hóa và làm sạch các comment mới
cleaned_comments = [normalize_text(comment) for comment in new_comments]

# Vectorize các comment mới bằng vectorizer đã huấn luyện
X_new_tfidf = vectorizer.transform(cleaned_comments)

# Dự đoán cảm xúc của các comment bằng mô hình kết hợp đã tinh chỉnh
y_pred_new = voting_classifier.predict(X_new_tfidf)

# Giải mã các nhãn dự đoán về các cảm xúc
predicted_labels = label_encoder.inverse_transform(y_pred_new)

# Hiển thị kết quả
for comment, predicted_label in zip(new_comments, predicted_labels):
    print(f'Comment: "{comment}"')
    print(f'Predicted Sentiment: {predicted_label}')
    print()
